<a href="https://colab.research.google.com/github/ZHANGZ1YUE/Rainfall_Nowcasting/blob/main/data_prep.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
#Data Preparation
#Author: ZHANG Ziyue (Kyoto University)
#Date: 2011/11/1

from google.colab import drive
drive.mount('/content/drive/')    
%cd /content/drive/MyDrive/Research/Data/20180704

Mounted at /content/drive/
/content/drive/MyDrive/Research/Data/20180704


In [2]:
import numpy as np
import glob
import matplotlib.pyplot as plt

In [17]:
base_dir = '/content/drive/MyDrive/Research/Data/20180704/conv_npy'
save_dir = '/content/drive/MyDrive/Research/30mins_4ch'

interval = 5                                            #5 minutes interval
channels = 4                                            #Taking 4 frames as input 
lead = 30                                               #30 mins prediction

In [ ]:
path = glob.glob(base_dir + '/*npy')     #for locating the path of the target files
path = [path[i] for i in range(len(path)) if i % interval == 0]    
print(len(path))

In [19]:
single_ch_ds = np.zeros((len(path), 180, 120, 1))                #Creating a nparray 4 dimension, first one indicates # sets of inputs
dataset = np.zeros((len(path)- (channels-1) - int(lead / interval), 180, 120, channels))      #Creating another box which takes information from single channel datasets, we will copy things in this dataset
                                                                 # i = 0,...,last-3, here-3是因为我们选取4个frame，最后4个frame会被放在一起（倒数第4,3,2,1个），如果再往后的话就是倒数第2个，第1个，没了！所以不行
                                                                 # 再-6是因为我们要用6*5=30分钟前的data去predict30分钟后的，所以
output = np.zeros((len(path)- (channels-1) - int(lead / interval), 180, 120, 1))                #对于output，我们只有一个channel，但是为了和input对齐，需要再加一个nparray dimension, 不然input4个dim，output3个dim

for i in range(len(path)):                                        # i = 0,...,81 一共82个，range即0-81
    single_ch_ds[i,:,:,0] = np.load(path[i])                      #At this moment, each single frame is a input in the dataset, and only 1st channel is assigned with a data, 2nd and 3ed remains 0

for i in range(len(path)- (channels-1) - int(lead / interval)):    
    k = i
    for j in range(channels):     # j = 0,1,2
        #print(k)   should give 0,1,2;   1,2,3;   2,3,4; 
        dataset[i,:,:,j] = single_ch_ds[k,:,:,0]                 #See my sticky note for expected structure
        k = k + 1


for i in range(len(path) - (channels-1) - int(lead / interval)):
    k = i + (channels-1) + int(lead / interval)
    output[i,:,:,0] = single_ch_ds[k,:,:,0]

In [ ]:
print(np.shape(single_ch_ds))
print(np.shape(dataset))
print(np.shape(output))

In [ ]:
np.save(save_dir+'/20180704input.npy', dataset)
np.save(save_dir+'/20180704output.npy', output)

In [ ]:
plt.imshow(output[40,:,:,0], cmap = 'hsv')

In [ ]:
plt.imshow(dataset[51,:,:,0], cmap = 'hsv')

In [ ]:
np.array_equal(output[0,:,:,0],dataset[6,:,:,3])  #should be equal